# GitHub on Hacker News trends analysis

In [1]:
from __future__ import print_function
import pandas as pd

## Input parameters

In [9]:
project_id = "apply-ds-test-322006"
process_date = "2021-08-01"
process_date_nodash = "20210801"

## Exploratory Data Analysis

## Github activity data
- Link: [Data](https://bigquery.cloud.google.com/table/githubarchive:day.20181230) - [More info](https://blog.github.com/2017-01-19-github-data-ready-for-you-to-explore-with-bigquery/)

### Different event type in Gihub activity
- [Event Types & Payloads](https://developer.github.com/v3/activity/events/types/) explaination

In [10]:
query = """
SELECT 
  type,
  COUNT(*) AS cnt
FROM `githubarchive.day.{0}` 
GROUP BY 1
ORDER BY 2 DESC
""".format(process_date_nodash)

print (query)

df = pd.read_gbq(query, project_id=project_id, dialect='standard')
df.head(20)


SELECT 
  type,
  COUNT(*) AS cnt
FROM `githubarchive.day.20210801` 
GROUP BY 1
ORDER BY 2 DESC



,type,cnt
0,PushEvent,1163742
1,CreateEvent,393430
2,PullRequestEvent,154756
3,WatchEvent,107482
4,IssueCommentEvent,94751
5,DeleteEvent,69254
6,IssuesEvent,39453
7,ForkEvent,37984
8,PullRequestReviewEvent,25388
9,PullRequestReviewCommentEvent,17126


### Top 10 repos with the most comments in their issues
- __IssueCommentEvent__: Triggered when an issue comment is created, edited, or deleted.

In [11]:
query = """
SELECT 
  repo.name,
  COUNT(*) AS cnt
FROM `githubarchive.day.{0}`
WHERE type IN ( 'IssueCommentEvent')
GROUP BY 1
ORDER BY 2 DESC
LIMIT 10
""".format(process_date_nodash)

print (query)

df = pd.read_gbq(query, project_id=project_id, dialect='standard')
df.head(20)


SELECT 
  repo.name,
  COUNT(*) AS cnt
FROM `githubarchive.day.20210801`
WHERE type IN ( 'IssueCommentEvent')
GROUP BY 1
ORDER BY 2 DESC
LIMIT 10



,name,cnt
0,flutter/flutter,1262
1,siputra12/fakecall,1083
2,microsoft/winget-pkgs,744
3,boost-e2e-stage-ci-buildkite/pr-comments,737
4,boost-e2e-tester-stage/non-main-pr,389
5,google-test/signcla-probe-repo,384
6,boost-e2e-stage-ci-buildkite/non-main-pr,327
7,openshift/assisted-service,286
8,NixOS/nixpkgs,272
9,hugoblanc/Athena,269


### Top 10 repos by stars and fork event

In [12]:
query = """
SELECT 
  repo.name,
  SUM(IF(type='WatchEvent', 1, NULL)) AS stars,
  SUM(IF(type='ForkEvent',  1, NULL)) AS forks,
  COUNT(*) AS cnt
FROM `githubarchive.day.{0}`
WHERE type IN ('WatchEvent','ForkEvent')
GROUP BY 1
ORDER BY 2 DESC
LIMIT 10
""".format(process_date_nodash)

print (query)

df = pd.read_gbq(query, project_id=project_id, dialect='standard')
df.head(20)


SELECT 
  repo.name,
  SUM(IF(type='WatchEvent', 1, NULL)) AS stars,
  SUM(IF(type='ForkEvent',  1, NULL)) AS forks,
  COUNT(*) AS cnt
FROM `githubarchive.day.20210801`
WHERE type IN ('WatchEvent','ForkEvent')
GROUP BY 1
ORDER BY 2 DESC
LIMIT 10



,name,stars,forks,cnt
0,electron/electron,427,7,434
1,myspaghetti/macos-virtualbox,304,5,309
2,freeCodeCamp/freeCodeCamp,277,23,300
3,bytebase/bytebase,255,3,258
4,cosmtrek/air,242,2,244
5,langjam/langjam,228,3,231
6,kunal-kushwaha/DSA-Bootcamp-Java,225,85,310
7,ibraheemdev/modern-unix,220,1,221
8,saadeghi/daisyui,210,1,211
9,mitmproxy/mitmproxy,204,14,218


## Hacker News data
- Link: [Data](https://bigquery.cloud.google.com/table/bigquery-public-data:hacker_news.full) - [More info](https://medium.com/@hoffa/hacker-news-on-bigquery-now-with-daily-updates-so-what-are-the-top-domains-963d3c68b2e2)

### Top domains shared in Hacker News
- Domain with higher score are more likely to make it to the front page.
- __nytimes__ has the highest average score.

In [13]:
query = """
SELECT 
  REGEXP_EXTRACT(url, '//([^/]*)/?') as domain,
  AVG(score) as avg_score,
  COUNT(*) AS cnt
FROM `bigquery-public-data.hacker_news.full`
WHERE url!='' 
AND EXTRACT(DATE FROM timestamp)="{0}"
GROUP BY 1
ORDER BY 3 DESC 
LIMIT 10
""".format(process_date)

print (query)

df = pd.read_gbq(query, project_id=project_id, dialect='standard')
df.head(20)


SELECT 
  REGEXP_EXTRACT(url, '//([^/]*)/?') as domain,
  AVG(score) as avg_score,
  COUNT(*) AS cnt
FROM `bigquery-public-data.hacker_news.full`
WHERE url!='' 
AND EXTRACT(DATE FROM timestamp)="2021-08-01"
GROUP BY 1
ORDER BY 3 DESC 
LIMIT 10



,domain,avg_score,cnt
0,www.youtube.com,4.225806,31
1,github.com,24.375000,24
2,en.wikipedia.org,3.894737,19
3,www.nytimes.com,9.642857,14
4,medium.com,19.000000,12
5,www.bbc.com,31.700000,10
6,twitter.com,22.222222,9
7,www.reuters.com,8.250000,8
8,www.wsj.com,3.250000,8
9,www.bbc.co.uk,33.428571,7


### What domains have the best chance of getting more than 40 upvotes?
- Certainly Hacker News likes content hosted on sites like github.com and the nytimes.

In [14]:
query = """
SELECT 
  REGEXP_EXTRACT(url, '//([^/]*)/?') as domain,
  COUNTIF(score>40) as score_gt_40,
  COUNT(*) AS cnt
FROM `bigquery-public-data.hacker_news.full`
WHERE url!='' 
AND EXTRACT(DATE FROM timestamp)="{0}"
GROUP BY 1
ORDER BY 2 DESC 
LIMIT 10
""".format(process_date)

print (query)

df = pd.read_gbq(query, project_id=project_id, dialect='standard')
df.head(20)


SELECT 
  REGEXP_EXTRACT(url, '//([^/]*)/?') as domain,
  COUNTIF(score>40) as score_gt_40,
  COUNT(*) AS cnt
FROM `bigquery-public-data.hacker_news.full`
WHERE url!='' 
AND EXTRACT(DATE FROM timestamp)="2021-08-01"
GROUP BY 1
ORDER BY 2 DESC 
LIMIT 10



,domain,score_gt_40,cnt
0,github.com,4,24
1,arstechnica.com,2,6
2,asiatimes.com,1,1
3,riowang.blogspot.com,1,1
4,fabiensanglard.net,1,1
5,blog.frantic.im,1,1
6,utcc.utoronto.ca,1,2
7,www.rechargenews.com,1,2
8,www.bbc.co.uk,1,7
9,www.latimes.com,1,1


### Top 10 Hacker news stories from Github by highest score

In [15]:
query = """
SELECT     
  `by` AS submitter,
  id as story_id,
  REGEXP_EXTRACT(url, "(https?://github.com/[^/]*/[^/#?]*)") as url,
  SUM(score) as score
FROM
  `bigquery-public-data.hacker_news.full`
WHERE
  type = 'story'
  AND EXTRACT(DATE FROM timestamp)='{0}' 
  AND url LIKE '%https://github.com%'
  AND url NOT LIKE '%github.com/blog/%'
GROUP BY  
  submitter,
  story_id,
  url
ORDER BY score DESC
""".format(process_date)

print (query)

df = pd.read_gbq(query, project_id=project_id, dialect='standard')
df.head(10)


SELECT     
  `by` AS submitter,
  id as story_id,
  REGEXP_EXTRACT(url, "(https?://github.com/[^/]*/[^/#?]*)") as url,
  SUM(score) as score
FROM
  `bigquery-public-data.hacker_news.full`
WHERE
  type = 'story'
  AND EXTRACT(DATE FROM timestamp)='2021-08-01' 
  AND url LIKE '%https://github.com%'
  AND url NOT LIKE '%github.com/blog/%'
GROUP BY  
  submitter,
  story_id,
  url
ORDER BY score DESC



,submitter,story_id,url,score
0,throwawaybutwhy,28025758,https://github.com/Satsuoni/widevine-l3-guesser,179
1,coldblues,28025491,https://github.com/org-roam/org-roam-ui,128
2,tinbucket,28028781,https://github.com/snowie2000/mactype,119
3,treesciencebot,28027016,https://github.com/isidentical/refactor,104
4,cactusbee,28030743,https://github.com/mozilla/contain-facebook,15
5,Borkdude,28025623,https://github.com/borkdude/nbb,5
6,Adam13531,28028293,https://github.com/Adam13531/homepagerizer,3
7,rachitnigam,28028043,https://github.com/rachitnigam/runt,3
8,tmpid,28031299,https://github.com/jonathandata1/pegasus_spyware,3
9,rtbtobi,28029682,https://github.com/rene-tobner/unity,3


## Example Final table: GitHub on Hacker News Trends of 2018-12-01

In [16]:
query = """
WITH github_activity AS (
SELECT   
  repo.name as repo,
  CONCAT('https://github.com/', repo.name) as url,
  SUM(IF(type='WatchEvent', 1, NULL)) AS stars,
  SUM(IF(type='ForkEvent',  1, NULL)) AS forks,
  COUNT(*) AS cnt
FROM `githubarchive.day.{0}`
WHERE type IN ('WatchEvent','ForkEvent')
GROUP BY 1,2
),
hacker_news AS (
SELECT
  EXTRACT(DATE FROM timestamp) as date,
  `by` AS submitter,
  id as story_id,
  REGEXP_EXTRACT(url, "(https?://github.com/[^/]*/[^/#?]*)") as url,
  SUM(score) as score
FROM
  `bigquery-public-data.hacker_news.full`
WHERE
  type = 'story'
  AND EXTRACT(DATE FROM timestamp)='{1}' 
  AND url LIKE '%https://github.com%'
  AND url NOT LIKE '%github.com/blog/%'
GROUP BY 1,2,3,4
)

SELECT
  a.date as date,
  a.url as github_url,
  b.repo as github_repo,
  a.score as hn_score,
  a.story_id as hn_story_id,
  b.stars as stars,
  b.forks as forks
FROM hacker_news as a
LEFT JOIN github_activity as b
ON a.url=b.url
ORDER BY hn_score DESC
LIMIT 10
""".format(process_date_nodash, process_date)

print (query)

df = pd.read_gbq(query, project_id=project_id, dialect='standard')
df.head(10)


WITH github_activity AS (
SELECT   
  repo.name as repo,
  CONCAT('https://github.com/', repo.name) as url,
  SUM(IF(type='WatchEvent', 1, NULL)) AS stars,
  SUM(IF(type='ForkEvent',  1, NULL)) AS forks,
  COUNT(*) AS cnt
FROM `githubarchive.day.20210801`
WHERE type IN ('WatchEvent','ForkEvent')
GROUP BY 1,2
),
hacker_news AS (
SELECT
  EXTRACT(DATE FROM timestamp) as date,
  `by` AS submitter,
  id as story_id,
  REGEXP_EXTRACT(url, "(https?://github.com/[^/]*/[^/#?]*)") as url,
  SUM(score) as score
FROM
  `bigquery-public-data.hacker_news.full`
WHERE
  type = 'story'
  AND EXTRACT(DATE FROM timestamp)='2021-08-01' 
  AND url LIKE '%https://github.com%'
  AND url NOT LIKE '%github.com/blog/%'
GROUP BY 1,2,3,4
)

SELECT
  a.date as date,
  a.url as github_url,
  b.repo as github_repo,
  a.score as hn_score,
  a.story_id as hn_story_id,
  b.stars as stars,
  b.forks as forks
FROM hacker_news as a
LEFT JOIN github_activity as b
ON a.url=b.url
ORDER BY hn_score DESC
LIMIT 10



,date,github_url,github_repo,hn_score,hn_story_id,stars,forks
0,2021-08-01,https://github.com/Satsuoni/widevine-l3-guesser,Satsuoni/widevine-l3-guesser,179,28025758,91.0,14.0
1,2021-08-01,https://github.com/org-roam/org-roam-ui,org-roam/org-roam-ui,128,28025491,156.0,6.0
2,2021-08-01,https://github.com/snowie2000/mactype,snowie2000/mactype,119,28028781,74.0,NaN
3,2021-08-01,https://github.com/isidentical/refactor,isidentical/refactor,104,28027016,74.0,NaN
4,2021-08-01,https://github.com/mozilla/contain-facebook,None,15,28030743,NaN,NaN
5,2021-08-01,https://github.com/borkdude/nbb,borkdude/nbb,5,28025623,16.0,NaN
6,2021-08-01,https://github.com/rene-tobner/unity,None,3,28029682,NaN,NaN
7,2021-08-01,https://github.com/rachitnigam/runt,None,3,28028043,NaN,NaN
8,2021-08-01,https://github.com/Adam13531/homepagerizer,Adam13531/homepagerizer,3,28028293,2.0,NaN
9,2021-08-01,https://github.com/antirez/pngtostl,antirez/pngtostl,3,28024537,31.0,1.0


## Python PyPI stats
- The Python Software Foundation provides the raw logs of Python installation activitie
- Link: [Data](https://bigquery.cloud.google.com/table/the-psf:pypi.downloads20181230) - [More info](https://packaging.python.org/guides/analyzing-pypi-package-downloads/)

__Challenge__: 
- Find associated Github stars, fork event, and Hacker News story for top downloads Python packages from pip

### Top 10 downloads packages from pip

In [15]:
query = """
SELECT 
  TIMESTAMP_TRUNC(timestamp, DAY) as day,
  file.project as project,
  COUNT(*) as downloads
FROM `the-psf.pypi.downloads{0}`
WHERE details.installer.name = 'pip'
GROUP BY 1,2
ORDER BY 3 desc
""".format(process_date_nodash)

print (query)

df = pd.read_gbq(query, project_id=project_id, dialect='standard')
df.head(10)


SELECT 
  TIMESTAMP_TRUNC(timestamp, DAY) as day,
  file.project as project,
  COUNT(*) as downloads
FROM `the-psf.pypi.downloads20181201`
WHERE details.installer.name = 'pip'
GROUP BY 1,2
ORDER BY 3 desc



,day,project,downloads
0,2018-12-01,pip,1562226
1,2018-12-01,urllib3,1271997
2,2018-12-01,botocore,1069194
3,2018-12-01,six,966172
4,2018-12-01,python-dateutil,946327
5,2018-12-01,s3transfer,877832
6,2018-12-01,docutils,813135
7,2018-12-01,pyyaml,796706
8,2018-12-01,pyasn1,782540
9,2018-12-01,jmespath,772065


### What is the number stars and fork event for botocore?

In [19]:
query = """
SELECT 
  repo.name,
  SUM(IF(type='WatchEvent', 1, NULL)) AS stars,
  SUM(IF(type='ForkEvent',  1, NULL)) AS forks,
  COUNT(*) AS cnt
FROM `githubarchive.day.{0}`
WHERE type IN ('WatchEvent','ForkEvent')
AND repo.name LIKE "%botocore%"
GROUP BY 1
ORDER BY 2 DESC
LIMIT 10
""".format(process_date_nodash)

print (query)

df = pd.read_gbq(query, project_id=project_id, dialect='standard')
df.head(20)


SELECT 
  repo.name,
  SUM(IF(type='WatchEvent', 1, NULL)) AS stars,
  SUM(IF(type='ForkEvent',  1, NULL)) AS forks,
  COUNT(*) AS cnt
FROM `githubarchive.day.20181201`
WHERE type IN ('WatchEvent','ForkEvent')
AND repo.name LIKE "%botocore%"
GROUP BY 1
ORDER BY 2 DESC
LIMIT 10



,name,stars,forks,cnt
0,boto/botocore,2,1,3


## Resources
- [GitHub data, ready for you to explore with BigQuery](https://blog.github.com/2017-01-19-github-data-ready-for-you-to-explore-with-bigquery/)
- [Hacker News on BigQuery](https://medium.com/@hoffa/hacker-news-on-bigquery-now-with-daily-updates-so-what-are-the-top-domains-963d3c68b2e2)
- [Analyzing PyPI package downloads](https://packaging.python.org/guides/analyzing-pypi-package-downloads/)